In [1]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities.sql_database import SQLDatabase
from langchain.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

import os
import time
import sys
import io

import nest_asyncio
nest_asyncio.apply()

In [6]:
# setup your OPENAI API key
OPENAI_API_KEY = ''

In [3]:
# load db
db = SQLDatabase.from_uri(f"sqlite:///sql_files/Chinook.db", sample_rows_in_table_info = 3)

In [5]:
# choose LLM Model
llm = ChatOpenAI(model='gpt-3.5-turbo', 
                temperature=0, 
                openai_api_key=OPENAI_API_KEY)

In [8]:
# create sql agent 
agent = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True, stream_runnable=False)

In [10]:
# chat with the database using sql agent
with get_openai_callback() as cb:
    result = agent.invoke("Which country's customers spent the most?")



> Entering new SQL Agent Executor chain...



Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice, InvoiceLine'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lima, 2170	São José dos Campos	SP	Brazil	12227-000	

In [11]:
# see the agent output
print("LLM Result: ", result['output'])

LLM Result:  Customers from the USA spent the most, with a total amount of $523.06.


In [12]:
# get the other call details
print(cb)

Tokens Used: 4377
	Prompt Tokens: 4272
	Completion Tokens: 105
Successful Requests: 4
Total Cost (USD): $0.0022935000000000004
